# GSPH Simulation Analysis Tutorial

This notebook demonstrates how to analyze GSPH simulation results using the analysis toolkit.

## Setup

First, ensure you have the required packages installed:
```bash
pip install numpy pandas scipy matplotlib
```

In [ ]:
# Import the analysis toolkit
import sys
sys.path.insert(0, '..')  # Add parent directory to path

from analysis import (
    SimulationReader,
    ConservationAnalyzer,
    TheoreticalComparison,
    ParticlePlotter,
    EnergyPlotter
)

import numpy as np
import matplotlib.pyplot as plt

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Load Simulation Data

Let's analyze a shock tube simulation (1D Riemann problem).

In [ ]:
# Specify the path to your simulation results
# Change this to your actual results directory
results_dir = '../results/shock_tube'

# Create a reader
reader = SimulationReader(results_dir)

# Print basic information
print(f"Simulation dimension: {reader.dim}D")
print(f"Number of snapshots: {reader.num_snapshots}")
print(f"Output files: {reader.snapshot_files[:5]}...")  # First 5

In [ ]:
# Read all snapshots
snapshots = reader.read_all_snapshots()

# Examine first snapshot
snap0 = snapshots[0]
print(f"\nFirst snapshot at t = {snap0.time:.4f}")
print(f"Number of particles: {snap0.num_particles}")
print(f"Position range: [{snap0.pos[:, 0].min():.3f}, {snap0.pos[:, 0].max():.3f}]")
print(f"Density range: [{snap0.dens.min():.3f}, {snap0.dens.max():.3f}]")
print(f"Total mass: {snap0.total_mass():.6f}")
print(f"Total kinetic energy: {snap0.total_kinetic_energy():.6f}")

## 2. Conservation Analysis

Check how well mass, momentum, and energy are conserved.

In [ ]:
# Analyze conservation
conservation = ConservationAnalyzer.analyze_snapshots(snapshots)

# Print summary
print("\n=== Conservation Analysis ===")
conservation.print_summary()

In [ ]:
# Plot conservation report
fig = EnergyPlotter.plot_conservation_report(conservation)
plt.tight_layout()
plt.show()

## 3. Energy Analysis

If energy.txt exists, we can track energy evolution in detail.

In [ ]:
# Try to read energy history
try:
    energy = reader.read_energy_history()
    
    print(f"\nEnergy history: {len(energy.time)} time steps")
    print(f"Initial total energy: {energy.total[0]:.6f}")
    print(f"Final total energy: {energy.total[-1]:.6f}")
    
    error = energy.relative_error()
    print(f"Max energy error: {max(abs(error)):.6e}")
    
    # Plot energy evolution
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))
    
    EnergyPlotter.plot_energy_history(energy, ax=axes[0], show_components=True)
    EnergyPlotter.plot_energy_error(energy, ax=axes[1])
    
    plt.tight_layout()
    plt.show()
    
except FileNotFoundError:
    print("\nNo energy.txt file found. Skipping energy analysis.")

## 4. Theoretical Comparison (Shock Tube)

Compare simulation with the analytical Sod shock tube solution.

In [ ]:
# For 1D simulations, compare with shock tube solution
if reader.dim == 1:
    # Use final snapshot
    snap_final = snapshots[-1]
    
    # Compare with theory
    gamma = 1.4  # Adiabatic index (adjust if different)
    x0 = 0.0     # Discontinuity position
    
    solution, error = TheoreticalComparison.compare_shock_tube(
        snap_final, 
        gamma=gamma, 
        x0=x0
    )
    
    print(f"\nTheoretical comparison at t = {snap_final.time:.4f}")
    print(f"L2 density error: {error:.6e}")
    
    # Plot comparison for all quantities
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    plotter = ParticlePlotter()
    quantities = ['dens', 'vel', 'pres', 'ene']
    titles = ['Density', 'Velocity', 'Pressure', 'Internal Energy']
    
    for i, (qty, title) in enumerate(zip(quantities, titles)):
        plotter.plot_1d(snap_final, quantity=qty, theory=solution, ax=axes[i])
        axes[i].set_title(f'{title} at t = {snap_final.time:.3f}')
    
    plt.tight_layout()
    plt.show()
else:
    print(f"\nShock tube comparison only available for 1D simulations.")
    print(f"Current simulation is {reader.dim}D.")

## 5. Evolution Visualization

Visualize how the system evolves over time.

In [ ]:
# Plot multiple snapshots
plotter = ParticlePlotter()

# Select a few snapshots to plot
n_plots = min(4, len(snapshots))
indices = np.linspace(0, len(snapshots)-1, n_plots, dtype=int)

fig, axes = plt.subplots(n_plots, 1, figsize=(12, 4*n_plots))
if n_plots == 1:
    axes = [axes]

for i, idx in enumerate(indices):
    snap = snapshots[idx]
    
    if reader.dim == 1:
        plotter.plot_1d(snap, quantity='dens', ax=axes[i])
        axes[i].set_title(f'Density at t = {snap.time:.3f}')
    elif reader.dim == 2:
        plotter.plot_2d_scatter(snap, quantity='dens', ax=axes[i])
        axes[i].set_title(f'Density at t = {snap.time:.3f}')
    else:
        # For 3D, plot a slice at z=0
        plotter.plot_3d_slice(
            snap, 
            quantity='dens',
            slice_axis=2,
            slice_position=0.0,
            ax=axes[i]
        )
        axes[i].set_title(f'Density (z=0 slice) at t = {snap.time:.3f}')

plt.tight_layout()
plt.show()

## 6. Error Evolution

Track how simulation error evolves over time.

In [ ]:
# For 1D shock tube, compute L2 error at each time
if reader.dim == 1:
    times = []
    errors = []
    
    for snap in snapshots:
        _, error = TheoreticalComparison.compare_shock_tube(
            snap,
            gamma=1.4,
            x0=0.0
        )
        times.append(snap.time)
        errors.append(error)
    
    # Plot error evolution
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(times, errors, 'o-', linewidth=2, markersize=6)
    ax.set_xlabel('Time')
    ax.set_ylabel('L2 Density Error')
    ax.set_title('Shock Tube Error Evolution')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nError statistics:")
    print(f"Min error: {min(errors):.6e}")
    print(f"Max error: {max(errors):.6e}")
    print(f"Final error: {errors[-1]:.6e}")

## 7. Particle Tracking

Follow specific particles through time.

In [ ]:
# Track a few particles
n_particles = snapshots[0].num_particles
particle_ids = [0, n_particles//4, n_particles//2, 3*n_particles//4]

# Extract trajectories
times = []
positions = {pid: [] for pid in particle_ids}
densities = {pid: [] for pid in particle_ids}

for snap in snapshots:
    times.append(snap.time)
    for pid in particle_ids:
        idx = np.where(snap.particle_id == pid)[0][0]
        positions[pid].append(snap.pos[idx, 0])
        densities[pid].append(snap.dens[idx])

# Plot particle histories
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Position vs time
for pid in particle_ids:
    axes[0].plot(times, positions[pid], 'o-', label=f'Particle {pid}', markersize=4)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Position')
axes[0].set_title('Particle Trajectories')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Density vs time
for pid in particle_ids:
    axes[1].plot(times, densities[pid], 'o-', label=f'Particle {pid}', markersize=4)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Density')
axes[1].set_title('Particle Density Evolution')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Create Animation

Generate an MP4 animation of the simulation.

**Note:** This requires ffmpeg to be installed.

In [ ]:
from analysis import AnimationMaker

# Create animation maker
maker = AnimationMaker(reader)

# Generate animation based on dimension
output_file = 'simulation_animation.mp4'

try:
    if reader.dim == 1:
        anim = maker.animate_1d(
            quantity='dens',
            output_file=output_file,
            fps=10,
            interval=1
        )
    elif reader.dim == 2:
        anim = maker.animate_2d(
            quantity='dens',
            output_file=output_file,
            fps=10,
            mode='scatter',  # Try 'grid' for interpolated visualization
            cmap='viridis'
        )
    else:
        print("3D animations not yet implemented in this notebook.")
        print("Use the make_animation.py script instead.")
    
    print(f"\nAnimation saved to: {output_file}")
    
except Exception as e:
    print(f"\nAnimation creation failed: {e}")
    print("Make sure ffmpeg is installed: brew install ffmpeg (macOS)")

## Summary

This notebook demonstrated:
1. ✅ Loading simulation data with `SimulationReader`
2. ✅ Checking conservation with `ConservationAnalyzer`
3. ✅ Analyzing energy evolution with `EnergyHistory`
4. ✅ Comparing with theory using `TheoreticalComparison`
5. ✅ Creating visualizations with `ParticlePlotter` and `EnergyPlotter`
6. ✅ Tracking individual particles
7. ✅ Generating animations with `AnimationMaker`

## Next Steps

- Try different simulations (Kelvin-Helmholtz, Evrard collapse, Sedov-Taylor)
- Modify analysis parameters (tolerance, grid resolution, colormap)
- Add custom theoretical solutions
- Compute additional derived quantities

For more details, see `analysis/README.md`.